# SQL Case Study Part 2

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

In [43]:
# First let's import os package to make sure that Python is 
#looking to the same dir where we're locally storing the "country_club DB" in:
import os
os.getcwd()

'C:\\Users\\18324\\Desktop\\SQL proj\\SQLFiles Tier 2'

In [44]:
os.chdir(r"C:\Users\18324\Desktop\SQL proj\SQLFiles Tier 2")

In [45]:
os.getcwd()

'C:\\Users\\18324\\Desktop\\SQL proj\\SQLFiles Tier 2'

In [46]:
os.listdir()

['.ipynb_checkpoints',
 'README.md',
 'sqlite_db_pythonsqlite.db',
 'SQLTasks Tier 2.sql']

In [47]:
#let's start the SQL engine that will communicate our queries to the database:

engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [17]:
table_names = engine.table_names()

print(table_names)

['Bookings', 'Facilities', 'Members']


In [18]:
members = pd.read_sql_query("SELECT * FROM Members", engine)
bookings = pd.read_sql_query("SELECT * FROM Bookings", engine)
facilities = pd.read_sql_query("SELECT * FROM Facilities", engine)

In [20]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [21]:
# We can now get a look at our data

members.head()
facilities.head()
bookings.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


# Produce a list of facilities with a total revenue less than 1000.The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [24]:
query = """
SELECT
t2.name AS Facility,
t2.Total_Revenue AS Total_Revenue
FROM
    (SELECT t1.name AS name,
    SUM(t1.revenue) AS Total_Revenue
    FROM
        (
        SELECT 
        b.bookid,
        f.name as name,
        CASE WHEN b.memid =0 THEN (b.slots * f.guestcost)
        ELSE (b.slots * f.membercost) END AS revenue
        FROM Bookings AS b
        LEFT JOIN Members AS m 
        USING (memid)
        LEFT JOIN Facilities AS f 
        USING (facid)) AS t1
    GROUP BY t1.name) as t2
GROUP BY Facility
HAVING Total_Revenue < 1000
ORDER BY Total_Revenue DESC;
"""
total_revenue = pd.read_sql_query(query, engine)
total_revenue

,Facility,Total_Revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


# Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [30]:
query = """
SELECT m1.surname|| ', ' ||m1.firstname AS Member,
CASE WHEN m1.recommendedby = m2.memid
THEN m2.surname|| ', ' || m2.firstname
ELSE NULL
END AS Recommender
FROM Members m1
INNER JOIN Members AS m2 ON m1.recommendedby = m2.memid
WHERE m1.recommendedby > 0
ORDER BY Member;
"""
recommended = pd.read_sql_query(query,engine)
recommended

,Member,Recommender
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


# Q12: Find the facilities with their usage by member, but not guests


In [50]:
query = """
SELECT
    f.name as Facility, 
    SUM(b.slots) AS Usage
FROM Bookings AS b
LEFT JOIN Facilities as f 
    ON f.facid = b.facid
LEFT JOIN Members as m 
    ON m.memid = b.memid
    WHERE b.memid != 0
GROUP BY Facility
ORDER BY Usage DESC;
"""

Usage = pd.read_sql_query(query, engine)
Usage

,Facility,Usage
0,Badminton Court,1086
1,Tennis Court 1,957
2,Massage Room 1,884
3,Tennis Court 2,882
4,Snooker Table,860
5,Pool Table,856
6,Table Tennis,794
7,Squash Court,418
8,Massage Room 2,54


# Find the facilities usage by month, but not guests

In [60]:
query = """
SELECT
    t1.month AS Month,
    t1.name as Facility, 
    SUM(t1.usage) AS Usage
FROM
    (
    SELECT strftime('%m', starttime) AS Month,
    f.name AS name,
    b.slots AS usage
    FROM Bookings as b
    LEFT JOIN Facilities as f 
        ON f.facid = b.facid
    LEFT JOIN Members as m
        ON m.memid = b.memid
    WHERE b.memid != 0) AS t1
GROUP BY Month, Facility
ORDER BY Month, Usage DESC;

"""

Usage = pd.read_sql_query(query, engine)
Usage

,Month,Facility,Usage
0,07,Tennis Court 1,201
1,07,Massage Room 1,166
2,07,Badminton Court,165
3,07,Snooker Table,140
4,07,Tennis Court 2,123
5,07,Pool Table,110
6,07,Table Tennis,98
7,07,Squash Court,50
8,07,Massage Room 2,8
9,08,Badminton Court,414
